# Coursera Capstone project

In [3]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

!pip install geopy
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values


import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

import folium # map rendering library

print('Libraries imported.')

You should consider upgrading via the 'c:\users\marta\anaconda3\python.exe -m pip install --upgrade pip' command.


Libraries imported.


### Introduction/Business Problem

The purpose of this project is to recommend a type of restaurant and area where to open it in the city center of Madrid. 

### Data that will be used

The data I am going to use is a list of zip codes of the city center that i have and then I will add the latitude and longitude. Besides I am going to get the venues data from Foursquare to know the categories of restaurants and cluster the area. Finally I will look for the type missing in each cluster.

Import the data of cps of Madrid

In [6]:
df=pd.read_excel("CPS_MADRID.xlsx")
df.head()

,CP
0,28001
1,28002
2,28003
3,28004
4,28005


Here I get the latitude and longitude

In [9]:
import geocoder

df['Latitude'] = df.apply(lambda row: geocoder.arcgis('{}, Spain'.format(row['CP'])).latlng[0], axis=1)
df['Longitude'] = df.apply(lambda row: geocoder.arcgis('{}, Spain'.format(row['CP'])).latlng[1], axis=1)

In [10]:
df.head()

,CP,Latitude,Longitude
0,28001,40.424870,-3.682793
1,28002,40.445035,-3.673343
2,28003,40.442985,-3.705943
3,28004,40.424445,-3.699007
4,28005,40.405755,-3.712972


Then I get the data from Foursquare

In [11]:
CLIENT_ID = 'RRSN0LZMUFW42FPOHT4UDESOBZE2OEMID2G45EN5YMYIIJ2K' # your Foursquare ID
CLIENT_SECRET = '1OK1F0EQXYJJMLDJG5QMMFVXF4HPLVVCXKHRZLCIK2WCZ4FG' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version
LIMIT = 100 # A default Foursquare API limit value

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: RRSN0LZMUFW42FPOHT4UDESOBZE2OEMID2G45EN5YMYIIJ2K
CLIENT_SECRET:1OK1F0EQXYJJMLDJG5QMMFVXF4HPLVVCXKHRZLCIK2WCZ4FG


In [12]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

madrid_venues = getNearbyVenues(names=df['CP'],
                                   latitudes=df['Latitude'],
                                   longitudes=df['Longitude'])

28001
28002
28003
28004
28005
28006
28007
28008
28009
28010
28011
28012
28013
28014
28015
28016
28017
28018
28019
28020
28021
28022
28023
28024
28025
28026
28027
28028
28029
28030
28031
28032
28033
28034
28035
28036
28037
28038
28039
28040
28041
28042
28043
28044
28045
28046
28047
28048
28049
28050
28051
28052
28053
28054
28055
28070
28071
28080
28082
28083
28085
28086
28087


In [13]:
madrid_venues.head()

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,28001,40.42487,-3.682793,La Cocina De Maria Luisa,40.423513,-3.681738,Spanish Restaurant
1,28001,40.42487,-3.682793,Amazónico,40.423838,-3.685025,Restaurant
2,28001,40.42487,-3.682793,MORE,40.423884,-3.683809,Restaurant
3,28001,40.42487,-3.682793,Numa,40.423327,-3.683640,Italian Restaurant
4,28001,40.42487,-3.682793,Federica & Co.,40.426082,-3.684854,Thrift / Vintage Store
